In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = torch.device("mps") if torch.backends.mps.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4


cpu


In [24]:
with open('Gita.txt','r', encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(set(text))
print(chars)
vocublary_size = len(chars)
print(vocublary_size)



['\n', ' ', '!', '"', '#', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
79


In [25]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([ 0,  0, 28, 57, 56, 61, 55, 53, 72, 61, 67, 66,  0,  0, 44, 39,  1, 33,
        38, 28, 33, 25,  0,  0, 43, 67,  1, 60, 53, 74, 57,  1, 33,  1, 70, 57,
        53, 56,  1, 72, 60, 61, 71,  1, 75, 67, 66, 56, 57, 70, 58, 73, 64,  1,
        53, 66, 56,  1, 71, 68, 61, 70, 61, 72, 10, 72, 60, 70, 61, 64, 64, 61,
        66, 59,  1, 71, 68, 57, 57, 55, 60,  9,  0, 26, 77,  1, 35, 70, 61, 71,
        60, 66, 53,  1, 53, 66, 56,  1, 40, 70])


In [26]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size]for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print("inputs:")
print(x)
print('targets:')
print(y)

tensor([82008, 58986, 17574, 75957])
inputs:
tensor([[75, 67, 70, 63, 71,  1, 75, 57],
        [10, 75, 61, 72, 60, 67, 73, 72],
        [57,  1, 71, 68, 57, 57, 55, 60],
        [66, 59,  1, 59, 64, 53, 66, 55]])
targets:
tensor([[67, 70, 63, 71,  1, 75, 57],
        [75, 61, 72, 60, 67, 73, 72],
        [ 1, 71, 68, 57, 57, 55, 60],
        [59,  1, 59, 64, 53, 66, 55]])


In [27]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([0]) target is tensor(0)
when input is tensor([0, 0]) target is tensor(28)
when input is tensor([ 0,  0, 28]) target is tensor(57)
when input is tensor([ 0,  0, 28, 57]) target is tensor(56)
when input is tensor([ 0,  0, 28, 57, 56]) target is tensor(61)
when input is tensor([ 0,  0, 28, 57, 56, 61]) target is tensor(55)
when input is tensor([ 0,  0, 28, 57, 56, 61, 55]) target is tensor(53)
when input is tensor([ 0,  0, 28, 57, 56, 61, 55, 53]) target is tensor(72)


In [28]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocublary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocublary_size, vocublary_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss  
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)

            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index_next = torch.cat((index, index_next),dim=1)
        return index

model = BigramLanguageModel(vocublary_size)
m = model.to(device)    

context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens= 500)[0].tolist())
print(generated_chars)


In [29]:
a = torch.rand(2,3,5)
print(a.shape)
x,y,z = a.shape
print(a.shape)
a = a.view(x,y,z)
print(x,y,z)
print(a.shape)



torch.Size([2, 3, 5])
torch.Size([2, 3, 5])
2 3 5
torch.Size([2, 3, 5])
